In [7]:
from utils import getLoaderDataset
from model import GPT
from train import train_model
import torch
from utils import generate
from model import GPT
import argparse


In [8]:
N = 64
B = 12
loader, dataset = getLoaderDataset(N, B, "./datasets/shakespear_corpus.txt")

In [9]:
print(len(dataset))

1003789


In [11]:
L = 12
d = 768
d_ff = 4 * d
h = 8
V = dataset.get_vocab_size()
model = GPT(B, L, d, d_ff, N, h, V)

In [ ]:
cpkt = torch.load("./runs/fat_model/model_6200.pt", map_location=torch.device('cpu'))
model.load_state_dict(cpkt)
print(model)

In [ ]:
new_tokens = generate(model, dataset.encode("Hello World"), 300)
print(dataset.decode(new_tokens))

In [15]:
N = 64
B = 12
L = 6
d = 128
d_ff = 4 * d
h = 6
V = dataset.get_vocab_size()

In [4]:
B = 12
N = 64  # context of up to 256 previous characters
L = 4
h = 4
d = 128
learning_rate = 1e-3
use_lr_decay = False
dataset = './datasets/shakespear_corpus.txt'
out_dir = './runs/'

max_iterations = 1000



class Args(argparse.Namespace):
  batch_size:int = B
  n_tokens:int = N
  n_layers:int = L
  n_heads:int = h
  d_model:int = d
  use_lr_decay:bool = use_lr_decay
  learning_rate: float = learning_rate
  dataset: str = dataset
  max_iterations: int = max_iterations
  out: str = out_dir

In [6]:
model, losses, perplexities = train_model(Args())

Epoch: 0, Batch 0, Training Loss: 4.387592792510986, Validation Loss: 3.8362371921539307
Epoch: 0, Batch 100, Training Loss: 2.5307981967926025, Validation Loss: 2.5348145961761475
Epoch: 0, Batch 200, Training Loss: 2.469670295715332, Validation Loss: 2.4643588066101074
Epoch: 0, Batch 300, Training Loss: 2.389496088027954, Validation Loss: 2.411836862564087
Epoch: 0, Batch 400, Training Loss: 2.4696075916290283, Validation Loss: 2.3877906799316406
Epoch: 0, Batch 500, Training Loss: 2.3984341621398926, Validation Loss: 2.360142946243286
Epoch: 0, Batch 600, Training Loss: 2.4015376567840576, Validation Loss: 2.3557770252227783
Epoch: 0, Batch 700, Training Loss: 2.2498743534088135, Validation Loss: 2.3132262229919434
Epoch: 0, Batch 800, Training Loss: 2.3662326335906982, Validation Loss: 2.274087429046631
Epoch: 0, Batch 900, Training Loss: 2.26304030418396, Validation Loss: 2.2421374320983887
Epoch: 0, Batch 1000, Training Loss: 2.1270015239715576, Validation Loss: 2.20957016944885

In [ ]:
import torch

B = 12
N = 64  # context of up to 256 previous characters
L = 4
h = 4
d = 128
learning_rate = 1e-3
betas = (0.9, 0.99)
eps = 10e-9
n_warmup_iterations = 100
learning_rate_decay_iterations = 5000
min_learning_rate = 1e-4
use_lr_decay = True
dataset = './datasets/shakespear_corpus.txt'

model = GPT(B, L, d, 3*d, N, h, V)

cpkt = torch.load("./runs/model_2000.pt", map_location=torch.device('cpu'))
model.load_state_dict(cpkt)
print(model)

In [ ]:
from utils import generate

new_tokens = generate(model, dataset.encode("Hello World"), 300)
print(dataset.decode(new_tokens))